# Decision Trees — Student Lab

We start using **sklearn** in Week 4, but you’ll still implement core pieces from scratch.

In [ ]:
import numpy as np

def check(name: str, cond: bool):
    if not cond:
        raise AssertionError(f'Failed: {name}')
    print(f'OK: {name}')

rng = np.random.default_rng(0)

## Section 0 — Synthetic dataset
We’ll create a non-linear boundary dataset to show how trees fit.

In [ ]:
def make_nonlinear(n=400):
    X = rng.uniform(-2, 2, size=(n, 2))
    # circle boundary
    r = np.sqrt(X[:,0]**2 + X[:,1]**2)
    y = (r < 1.0).astype(int)
    # add noise
    flip = rng.random(n) < 0.05
    y[flip] = 1 - y[flip]
    return X, y

X, y = make_nonlinear()
n = X.shape[0]
idx = rng.permutation(n)
tr = idx[: int(0.7*n)]
va = idx[int(0.7*n):]
Xtr, ytr = X[tr], y[tr]
Xva, yva = X[va], y[va]
check('shapes', Xtr.shape[0]==ytr.shape[0] and Xva.shape[0]==yva.shape[0])

## Section 1 — Impurity

### Task 1.1: Gini impurity

# TODO: implement gini(y)
# HINT: p_k = count_k / n; gini = 1 - sum(p_k^2)


In [ ]:
def gini(y):
    # TODO
    ...

check('gini_pure0', abs(gini(np.zeros(10, dtype=int))) < 1e-12)
check('gini_half', abs(gini(np.array([0,1]*5)) - 0.5) < 1e-12)

### Task 1.2: Entropy

# TODO: implement entropy(y)
# HINT: entropy = -sum p log2 p (use eps)


In [ ]:
def entropy(y):
    # TODO
    ...

check('entropy_pure0', abs(entropy(np.zeros(10, dtype=int))) < 1e-12)
check('entropy_half', abs(entropy(np.array([0,1]*5)) - 1.0) < 1e-9)

## Section 2 — Best split (decision stump)

### Task 2.1: Evaluate impurity after threshold split

Split rule: go left if X[:,j] <= t else right.
Return weighted impurity and information gain.


In [ ]:
def split_indices(X, j, t):
    left = np.where(X[:, j] <= t)[0]
    right = np.where(X[:, j] > t)[0]
    return left, right

def info_gain(y, y_left, y_right, criterion='gini'):
    # TODO
    ...

# quick sanity
y0 = np.array([0,0,1,1])
gain = info_gain(y0, np.array([0,0]), np.array([1,1]), criterion='gini')
check('gain_positive', gain > 0)

### Task 2.2: Find best (feature, threshold)

# TODO: implement best_split(X, y)
# HINT: thresholds from sorted unique feature values midpoints

**FAANG gotcha:** if a split makes an empty child, skip it.

In [ ]:
def best_split(X, y, criterion='gini'):
    # TODO: return (best_j, best_t, best_gain)
    ...

j, t, gain = best_split(Xtr, ytr)
print('best', j, t, gain)
check('gain_nonneg', gain >= 0)

### Task 2.3: Train a stump and evaluate

Use best_split to build a stump that predicts majority class on each side.


In [ ]:
def stump_predict(X_train, y_train, X_test, criterion='gini'):
    # TODO
    ...

def accuracy(y, yhat):
    return float(np.mean(y == yhat))

yhat_tr = stump_predict(Xtr, ytr, Xtr)
yhat_va = stump_predict(Xtr, ytr, Xva)
print('stump train acc', accuracy(ytr, yhat_tr))
print('stump val acc', accuracy(yva, yhat_va))

## Section 3 — sklearn DecisionTreeClassifier (sanity check)

### Task 3.1: Train trees with different max_depth

# TODO: train sklearn tree and compare train/val accuracy for depth in [1,2,3,5,None].


In [ ]:
from sklearn.tree import DecisionTreeClassifier

depths = [1,2,3,5,6,7,None]
for md in depths:
    clf = DecisionTreeClassifier(max_depth=md, random_state=0)
    clf.fit(Xtr, ytr)
    tr_acc = clf.score(Xtr, ytr)
    va_acc = clf.score(Xva, yva)
    print('max_depth', md, 'train', tr_acc, 'val', va_acc)

## Section 4 — Failure mode: leakage

### Task 4.1: Create a leaky feature
Add a feature that is directly derived from y and watch validation accuracy jump.

**Explain:** why do trees exploit leakage aggressively?

In [ ]:
Xtr_leak = np.hstack([Xtr, ytr.reshape(-1,1)])
Xva_leak = np.hstack([Xva, yva.reshape(-1,1)])

clf = DecisionTreeClassifier(max_depth=3, random_state=0)
clf.fit(Xtr_leak, ytr)
print('val acc with leakage', clf.score(Xva_leak, yva))

---
## Submission Checklist
- All TODOs completed
- Stump implemented
- sklearn depth sweep shown
- Leakage demo explained
